In [4]:
import numpy as np
from pymoo.algorithms.nsga2 import NSGA2
from pymoo.factory import get_problem,get_termination
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter
from pymoo.model.problem import Problem
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random

In [26]:
#defining problem
class AdaptationProblem(Problem):
    """
    response times are filled based on papers values(page8-table1)
    """
    def __init__(self,
                landa=50, #arrival rate (req/s)
                n=10, #average data payload for each request (KB)
                p_i=2.64E-04, #cost of a VM ($/s)
                p_n=3.50E-06, #cost of data transfer ($/KB)
                H=8.50E-05, #static Hosting cost ($/s)
                RPM=0.7, #revenue per 1000 ads ($)
                R=100, #response time (ms)
                gamma_l=1, #gamma lower bound (gamma is average number of ad banners per page)
                gamma_u=10, #gamma upper bound
                R_l=45, # respose time lower bound
                R_u=200, # response time upper bound
                d_l=15, #capacity of each VM lower bound(request/s)
                d_u=21 #capacity of each VM upper bound(request/s)
                ): 
        self.landa = landa
        self.n = n
        self.p_i = p_i
        self.p_n = p_n
        self.H = H
        self.RPM = RPM
        self.R = R
        self.gamma_l = gamma_l
        self.gamma_u = gamma_u
        self.R_l = R_l
        self.R_u = R_u
        self.d_l = d_l
        self.d_u = d_u
        #weights in user satisfaction formula
        self.a = 0.5
        self.b = 0.5
        #p_s doesn't have any bound so we should feed algorithm 
        #a very high number for upper bound and 0 (logically) for lower bound
        super().__init__(n_var=3, #p_s,W,gamma
                         n_obj=3, #pi_s,pi_a,U
                         n_constr=4,
                         xl=np.array([0,landa/d_u,gamma_l]),
                         xu=np.array([10000,landa/d_l,gamma_u]))

    def _evaluate(self, x, out, *args, **kwargs):
        #objectives
        #to maximize objectives we multipy them by -1 and then minimize them 
        #objectives are not normalized. test normalization on results
        d = self.landa/x[:,1]
        rel_d = (d - self.d_l) / (self.d_u - self.d_l)
        R = ((self.R_u-self.R_l)*rel_d) + self.R_l
        pi_s = x[:,0]*self.landa - self.p_i*x[:,1] - self.n*self.p_n*self.landa
        pi_a = (x[:,2]*self.RPM/1000)*self.landa - self.H*self.landa - x[:,0]*self.landa
        U = self.a*((self.gamma_u-x[:,2])/(self.gamma_u-self.gamma_l)) + self.b*((self.R_u-R)/(self.R_u-self.R_l))
        
        out["F"] = np.column_stack([-1*pi_s, -1*pi_a,-1*U])
            
        #constrains (equations are filped in a way that they would be less than zero)
        g1 = x[:,0] - (x[:,2]*self.RPM/1000)
        g2 = -1*(pi_s)
        g3 = -1*(pi_a)
        g4 = (0.8 - U)

        out["G"] = np.column_stack([g1, g2, g3, g4])


In [27]:
#defining algorithm
algorithm = NSGA2(
    pop_size=100,
    n_offsprings=10,
    eliminate_duplicates=True
)

In [28]:
#defining termination
termination = get_termination('n_gen',150)

In [45]:
#running minimize problem
problem = AdaptationProblem(R=1919.414,n=589.0,landa=52.1)

res = minimize(problem,
               algorithm,
               termination,
               save_history=True,
               verbose=True
              )

n_gen |  n_eval |   cv (min)   |   cv (avg)   |  n_nds  | delta_ideal  | delta_nadir  |   delta_f   
    1 |     100 |  1.75636E+03 |  2.62914E+05 |       1 |            - |            - |            -
    2 |     110 |  1.24885E+03 |  2.34504E+05 |       1 |  4.97912E+02 |  4.97912E+02 |  7.04132E+32
    3 |     120 |  1.24885E+03 |  2.07112E+05 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
    4 |     130 |  1.24885E+03 |  1.76444E+05 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
    5 |     140 |  1.24885E+03 |  1.50560E+05 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
    6 |     150 |  1.24885E+03 |  1.25953E+05 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
    7 |     160 |  1.24885E+03 |  1.06179E+05 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
    8 |     170 |  1.24885E+03 |  8.56880E+04 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
    9 |     180 |  1.24885E+03 |  7.07082E+04 |       1 |  0.00000E+00 |  0.00000E+00 |  0.

  125 |    1340 |  0.008768471 |  0.024723255 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
  126 |    1350 |  0.008768471 |  0.024153446 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
  127 |    1360 |  0.008768471 |  0.023353852 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
  128 |    1370 |  0.008768471 |  0.022651403 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
  129 |    1380 |  0.008768471 |  0.021415488 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
  130 |    1390 |  0.008768471 |  0.020325977 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
  131 |    1400 |  0.008768471 |  0.019449413 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
  132 |    1410 |  0.008768471 |  0.018511770 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
  133 |    1420 |  0.008768471 |  0.017729057 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
  134 |    1430 |  0.008768471 |  0.016859901 |       1 |  0.00000E+00 |  0.00000E+00 |  0.

In [46]:
res.X

In [ ]:



fig = pyplot.figure()
ax = Axes3D(fig)
f = -1*res.F

sequence_containing_x_vals = f[:,0]
sequence_containing_y_vals = f[:,1]
sequence_containing_z_vals = f[:,2]

random.shuffle(sequence_containing_x_vals)
random.shuffle(sequence_containing_y_vals)
random.shuffle(sequence_containing_z_vals)

ax.scatter(sequence_containing_x_vals, sequence_containing_y_vals, sequence_containing_z_vals)
pyplot.show()

In [ ]:
# collect the population in each generation
pop_each_gen = [a.pop for a in res.history]

# receive the population in each generation
obj_and_feasible_each_gen = [pop[pop.get("feasible")[:,0]].get("F") for pop in pop_each_gen]


In [ ]:
X = np.array([pop.get("X") for pop in pop_each_gen])
G = np.array([pop.get("G") for pop in pop_each_gen])
F = np.array([pop.get("F") for pop in pop_each_gen])

In [ ]:
gen = 129

In [ ]:
plt.scatter(G[gen][:,0],G[gen][:,1])
plt.xlabel("p_s - y*RPM/1000")
plt.ylabel("-pi_s")

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(G[gen][:,2],G[gen][:,3])
plt.xlabel("-pi_a")
plt.ylabel("0.8-U")

In [ ]:
plt.scatter(X[gen][:,0],X[gen][:,1])
plt.xlabel("p_s")
plt.ylabel("W")

In [ ]:
plt.scatter(X[gen][:,1],X[gen][:,2])
plt.xlabel("W")
plt.ylabel("gamma")

In [ ]:
pr = AdaptationProblem(landa=250,n=0.1,R_l=1,R_u=200,d_l=10,d_u=200)

In [ ]:
res = minimize(pr,
               algorithm,
               termination,
               save_history=True,
               verbose=True
              )

In [ ]:

def choose_on_pf(pareto_f,design_space,w_s=0.33,w_a=0.33,w_u=0.33):
    max_values = np.max(pareto_f,axis=0)
    arg_max = np.argmax(pareto_f[:,0]*w_s/max_values[0] + pareto_f[:,1]*w_a/max_values[1] + pareto_f[:,2]*w_u/max_values[2])
    return tuple(design_space[arg_max])

In [ ]:
p_s , W , gamma = choose_on_pf(-1*res.F,res.X)